## Initial preparation

In [1]:
import sys

sys.path.append('/garmentor/')

from vis.visualizers.keypoints import KeypointsVisualizer
from data.generate.pregenerator import DataPreGenerator

## Random parameters

In [2]:
data_generator = DataPreGenerator()
pose, shape, cam_t, _ = data_generator.generate_random_params()

## Visualize keypoints

In [3]:
keypoints_visualizer = KeypointsVisualizer()
keypoints_visualizer.vis_from_params(
    pose=pose,
    shape=shape,
    cam_t=cam_t
)

ValueError: operands could not be broadcast together with shapes (17,3) (1,4,4) 